# Atividade 6: Inferência de Dados e Apresentação de Resultados

## 1. Introdução

Este relatório apresenta a análise exploratória e os testes de hipóteses realizados sobre o dataset "The impact of artificial intelligence on society" do Kaggle, com foco na percepção pública sobre a Inteligência Artificial no mercado de trabalho e na sociedade.

**Grupo:** Larissa Beatriz, Vinícius Fernandes, Pedro Enrico e Pedro Moura

### Hipóteses a serem testadas:

1. H1 (Idade): Pessoas mais velhas confiam menos na inteligência artificial do que os jovens.

2. H2 (Emprego): A maioria das pessoas que não confiam na IA acreditam que a IA pode roubar seus empregos.

3. H3 (Escolaridade): Pessoas com maior nível de escolaridade tendem a perceber a IA de forma mais positiva.

4. H4 (Uso Prévio): Indivíduos que já utilizaram ferramentas de IA (proxy: maior conhecimento) demonstram maior confiança na tecnologia.

5. H5 (Ameaça/Setor): A percepção de que a IA é uma ameaça aumenta entre pessoas que trabalham em setores mais suscetíveis à automação (proxy: fatores como escolaridade e confiança predizem a percepção de ameaça ao próprio emprego).

## 2. Metodologia

| Hipótese | Variáveis Envolvidas | Métrica (Dependente) | Teste Estatístico | Conclusão |
|----------|-----------------------|----------------------|-------------------|------------|
| H1 e H4  | Idade vs. Confiança; Conhecimento vs. Confiança | Confiança (Escala 1-5) | Mann-Whitney U | Diferença entre 2 grupos |
| H2       | Confiança vs. Medo de Eliminação de Profissões | Frequência de Respostas | Qui-Quadrado | Comparação de proporções |
| H3       | Escolaridade (5 grupos) | Confiança (Escala 1-5) | Kruskal-Wallis | Comparação entre 3+ grupos |
| H5       | Confiança + Escolaridade | Ameaça ao Emprego (Escala 1-5) | Regressão Linear (OLS) | Analisar fatores de influência |

## 3. Resultados (Análise Estatística)

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.linear_model import LinearRegression

# ==============================================================================
# 1. CARREGAMENTO E PRÉ-PROCESSAMENTO BASE
# ==============================================================================
df = pd.read_csv('the_impact_of_artificial_intelligence_on_society.csv', encoding='latin1')

# Renomear colunas essenciais
df.rename(columns={
    'What is your age range?': 'Idade',
    'What is your education level?': 'Escolaridade',
    'Do you generally trust artificial intelligence (AI)?': 'Confianca',
    'Could artificial intelligence (AI) completely eliminate some professions?': 'Eliminar_Profissoes',
    'How much knowledge do you have about artificial intelligence (AI) technologies?': 'Conhecimento_IA', 
    'Do you think your own job could be affected by artificial intelligence (AI)?': 'Ameaca_Emprego', 
}, inplace=True)

# Mapeamento numérico da Confiança (Escala Likert 1 a 5)
confianca_map = {
    "I don't trust it at all": 1, "I don't trust it": 2, "I'm undecided": 3, "I trust it": 4, "I fully trust it": 5
}
df['Confianca_Numerica'] = df['Confianca'].map(confianca_map)
df.dropna(subset=['Confianca_Numerica'], inplace=True)

alpha = 0.05 # Nível de significância padrão

In [ ]:
# ==============================================================================
# H1: IDADE vs. CONFIANÇA
# ==============================================================================
print("\n" + "="*50 + "\nH1: IDADE vs. CONFIANÇA\n" + "="*50)

def categorizar_idade(faixa):
    if faixa in ['Under 18 years old', '18-24', '25-34']:
        return 'Jovens'
    elif faixa in ['35-44', '45 years and older']:
        return 'Mais Velhos'
    return np.nan 

df['Grupo_Idade'] = df['Idade'].apply(categorizar_idade)
df_h1 = df.dropna(subset=['Grupo_Idade', 'Confianca_Numerica']).copy()
vendas_jovens = df_h1[df_h1['Grupo_Idade'] == 'Jovens']['Confianca_Numerica']
vendas_velhos = df_h1[df_h1['Grupo_Idade'] == 'Mais Velhos']['Confianca_Numerica']
descritivas_h1 = df_h1.groupby('Grupo_Idade')['Confianca_Numerica'].agg(['count', 'mean', 'std']).round(2)
print("Estatísticas Descritivas H1:\n", descritivas_h1)

u_stat, p_value_h1 = stats.mannwhitneyu(vendas_jovens, vendas_velhos, alternative='less') 
print(f"Resultado Mann-Whitney U (H1): Valor-p = {p_value_h1:.4f}")
print(f"Conclusão H1: {'SUPORTADA' if p_value_h1 < alpha else 'NÃO SUPORTADA'}")

plt.figure(figsize=(8, 6))
sns.boxplot(x='Grupo_Idade', y='Confianca_Numerica', data=df_h1)
plt.title('Confiança em IA por Grupo de Idade (H1)')
plt.savefig('boxplot_confianca_idade_final.png')


# ==============================================================================
# H2: CONFIANÇA vs. ELIMINAÇÃO DE PROFISSÕES
# ==============================================================================
print("\n" + "="*50 + "\nH2: CONFIANÇA vs. MEDO DO EMPREGO\n" + "="*50)

def simplificar_confianca(conf):
    if conf in ["I don't trust it at all", "I don't trust it"]: return '1_Nao_Confia'
    elif conf in ["I trust it", "I fully trust it"]: return '3_Confia'
    return '2_Indeciso'

def simplificar_eliminacao(elim):
    if elim in ['Definitely Removes', 'Removes']: return '1_Acredita_Eliminacao'
    elif elim in ['Definitely Doesn\'t Remove', 'Doesn\'t Remove']: return '3_Nao_Acredita_Eliminacao'
    return '2_Indeciso'

df['Confianca_Grupo'] = df['Confianca'].apply(simplificar_confianca)
df['Eliminacao_Grupo'] = df['Eliminar_Profissoes'].apply(simplificar_eliminacao)
df_h2 = df.dropna(subset=['Confianca_Grupo', 'Eliminacao_Grupo']).copy()

contingency_table = pd.crosstab(df_h2['Confianca_Grupo'], df_h2['Eliminacao_Grupo'], normalize='index')
contingency_counts = pd.crosstab(df_h2['Confianca_Grupo'], df_h2['Eliminacao_Grupo'])

chi2, p_value_h2, dof, expected = stats.chi2_contingency(contingency_counts)
print("Tabela de Contingência H2 (Percentual de Acreditar em Eliminação):\n", contingency_table.mul(100).round(1).to_markdown(numalign="left", stralign="left"))
print(f"Resultado Qui-Quadrado (H2): Valor-p = {p_value_h2:.4f}")
print(f"Conclusão H2: {'SUPORTADA' if p_value_h2 < alpha else 'NÃO SUPORTADA'}")

plt.figure(figsize=(10, 6))
contingency_table.plot(kind='bar', stacked=True)
plt.title('Opinião sobre Eliminação de Empregos por Nível de Confiança (H2)')
plt.savefig('chi2_confianca_empregos.png')


# ==============================================================================
# H3: ESCOLARIDADE vs. CONFIANÇA
# ==============================================================================
print("\n" + "="*50 + "\nH3: ESCOLARIDADE vs. CONFIANÇA\n" + "="*50)

df_h3 = df.dropna(subset=['Escolaridade', 'Confianca_Numerica']).copy()
grupos_escolaridade = df_h3['Escolaridade'].unique().tolist()
grupos_de_vendas = [df_h3[df_h3['Escolaridade'] == g]['Confianca_Numerica'] for g in grupos_escolaridade]

descritivas_h3 = df_h3.groupby('Escolaridade')['Confianca_Numerica'].agg(['count', 'mean', 'std']).round(2)
print("Estatísticas Descritivas H3:\n", descritivas_h3.sort_values(by='mean', ascending=False))

kruskal_stat, p_value_h3 = stats.kruskal(*grupos_de_vendas)
print(f"Resultado Kruskal-Wallis (H3): Valor-p = {p_value_h3:.4f}")
print(f"Conclusão H3: {'SUPORTADA' if p_value_h3 < alpha else 'NÃO SUPORTADA'}")

plt.figure(figsize=(12, 7))
sns.boxplot(x='Escolaridade', y='Confianca_Numerica', data=df_h3, order=descritivas_h3.sort_values(by='mean', ascending=False).index)
plt.title('Confiança em IA por Nível de Escolaridade (H3)')
plt.savefig('boxplot_confianca_escolaridade.png')

In [ ]:
# ==============================================================================
# H4: CONHECIMENTO IA vs. CONFIANÇA (Mann-Whitney U)
# ==============================================================================
print("\n" + "="*50 + "\nH4: CONHECIMENTO IA vs. CONFIANÇA\n" + "="*50)

def categorizar_conhecimento(nivel):
    if nivel in ['I have a good level of knowledge', 'I have basic knowledge']: return '1_Alto_Medio_Conhecimento'
    elif nivel in ['I\'ve heard a little about it', 'I don\'t know anything']: return '2_Baixo_Nenhum_Conhecimento'
    return np.nan 

df['Grupo_Conhecimento'] = df['Conhecimento_IA'].apply(categorizar_conhecimento)
df_h4 = df.dropna(subset=['Grupo_Conhecimento', 'Confianca_Numerica']).copy()
vendas_alto = df_h4[df_h4['Grupo_Conhecimento'] == '1_Alto_Medio_Conhecimento']['Confianca_Numerica']
vendas_baixo = df_h4[df_h4['Grupo_Conhecimento'] == '2_Baixo_Nenhum_Conhecimento']['Confianca_Numerica']

descritivas_h4 = df_h4.groupby('Grupo_Conhecimento')['Confianca_Numerica'].agg(['count', 'mean', 'std']).round(2)
print("Estatísticas Descritivas H4:\n", descritivas_h4.to_markdown(numalign="left", stralign="left"))

u_stat, p_value_h4 = stats.mannwhitneyu(vendas_alto, vendas_baixo, alternative='greater')
print(f"Resultado Mann-Whitney U (H4): Valor-p = {p_value_h4:.4f}")
print(f"Conclusão H4: {'SUPORTADA' if p_value_h4 < alpha else 'NÃO SUPORTADA'}")

plt.figure(figsize=(8, 6))
sns.boxplot(x='Grupo_Conhecimento', y='Confianca_Numerica', data=df_h4)
plt.title('Confiança em IA por Nível de Conhecimento (H4)')
plt.savefig('boxplot_confianca_conhecimento.png')
print("Boxplot salvo como 'boxplot_confianca_conhecimento.png'")


# ==============================================================================
# H5: REGRESSÃO LINEAR (FATORES DE AMEAÇA AO EMPREGO)
# ==============================================================================
print("\n" + "="*50 + "\nH5: REGRESSÃO LINEAR\n" + "="*50)

# 1. Mapeamento e criação da variável dependente numérica (Y)
ameaca_map = {
    'Definitely don\'t think': 1, 'Don\'t think': 2, 'I\'m undecided': 3, 'Think': 4, 'I definitely think': 5
}
df['Ameaca_Emprego_Numerica'] = df['Ameaca_Emprego'].map(ameaca_map)

# 2. SELEÇÃO E LIMPEZA RIGOROSA DO DATAFRAME DA REGRESSÃO
# ODS exige que TODAS as variáveis de X e Y estejam preenchidas
df_reg = df[['Ameaca_Emprego_Numerica', 'Confianca_Numerica', 'Escolaridade']].copy()
df_reg.dropna(inplace=True) # AQUI ESTÁ A CHAVE: Limpar o DF antes de criar as Dummies

# 3. PREPARAÇÃO DE VARIÁVEIS PREDITORAS (X)
# Criar variáveis dummy (One-Hot Encoding) para 'Escolaridade'
df_encoded = pd.get_dummies(df_reg, columns=['Escolaridade'], drop_first=True, dtype=int) 

# Selecionar variáveis Y e X
Y = df_encoded['Ameaca_Emprego_Numerica']
X_cols = ['Confianca_Numerica'] + [col for col in df_encoded.columns if col.startswith('Escolaridade_')]
X = df_encoded[X_cols]

# Adicionar constante para o modelo OLS
X = sm.add_constant(X)

# 4. A LINHA CRÍTICA: OLS Regression
try:
    modelo = sm.OLS(Y, X).fit()
    print("Resultados do Modelo de Regressão Linear (OLS) H5:")
    print(modelo.summary())

    # Visualização da relação Confiança vs. Ameaça
    plt.figure(figsize=(10, 6))
    sns.regplot(x='Confianca_Numerica', y='Ameaca_Emprego_Numerica', data=df_reg, line_kws={"color": "red"})
    plt.title('Regressão Linear: Confiança vs. Ameaça ao Emprego (H5)')
    plt.savefig('regressao_ameaca_emprego.png')
    print("Gráfico de Regressão salvo como 'regressao_ameaca_emprego.png'")

except ValueError as e:
    print(f"Erro no modelo OLS: {e}. Verifique se as colunas 'Escolaridade' e 'Ameaca_Emprego' não tinham valores estranhos após o carregamento.")

print("="*100)

## 4. Conclusão Final do Relatório (Interpretação)
Esta seção deve ser escrita em Markdown no seu notebook, após o código.

A análise estatística das cinco hipóteses resultou em descobertas cruciais sobre a percepção da IA na sociedade:

1. H1 (Idade vs. Confiança): NÃO SUPORTADA.

    - Não houve diferença estatisticamente significativa na média de confiança entre jovens ( 
x
ˉ
 =3.07) e mais velhos ( 
x
ˉ
 =2.86). Isso sugere que a confiança na IA é um fator cultural ou de familiaridade, não estritamente etário.

2. H2 (Medo do Emprego vs. Confiança): NÃO SUPORTADA.

    - O Teste Qui-Quadrado (p=0.9770) mostrou que a crença na eliminação de profissões é homogênea em todos os grupos de confiança (cerca de 77% em todos os grupos). O medo da automação é um temor social generalizado, independente da postura individual de confiança.

3. H3 (Escolaridade vs. Confiança): NÃO SUPORTADA.

    - O Kruskal-Wallis (p=0.9371) indicou que o nível de escolaridade não tem impacto estatisticamente significativo na confiança em IA.

4. H4 (Uso Prévio vs. Confiança): SUPORTADA.

    - O Teste de Mann-Whitney U (p=0.0044) confirmou que indivíduos com Alto/Médio Conhecimento em IA ( 
x
ˉ
 =3.08) demonstram uma confiança significativamente maior do que aqueles com Baixo/Nenhum Conhecimento ( 
x
ˉ
 =2.68). Isso destaca a importância da familiaridade e educação prática para a aceitação da tecnologia.

5. H5 (Regressão Linear): PARCIALMENTE SUPORTADA E CONTRA-INTUITIVA.

    - O modelo de Regressão Linear é significativo (Prob(F-statistic) ≈1.03e−05) e explica 20.6% da variância na Ameaça ao Emprego.
    - O coeficiente de Confiança não é estatisticamente significativo (p=0.078).
    - O ACHADO MAIS FORTE: As variáveis de Menor Escolaridade (Ensino Fundamental e Ensino Médio) estão associadas a uma menor percepção de ameaça ao próprio emprego (coeficientes negativos significativos). Este resultado desafia a narrativa comum de que empregos de baixa qualificação são os mais ameaçados e sugere que a percepção de ameaça é maior entre aqueles com Graduação/Pós-Graduação, que talvez estejam mais cientes da capacidade da IA de automatizar tarefas intelectuais.